# 1. Write a python program which searches all the product under a particular product from www.amazon.in. The product to be searched will be taken as input from user. For e.g. If user input is ‘guitar’. Then search for guitars. 

In [45]:
# Importing necessary libraries.

from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import re
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By


import warnings
warnings.filterwarnings('ignore')

In [46]:
driver=webdriver.Chrome(r"chromedriver.exe")

In [47]:
driver.get("http://www.amazon.in/")

In [49]:
element=input("Searching for :")      #for input method
search=driver.find_elements(By.ID,"twotabsearchtextbox")  #search field
search[0].send_keys(element)                              #enter keys for searching items
time.sleep(3)

Searching for :Laptop


In [50]:
search=driver.find_element(By.XPATH,'/html/body/div[1]/header/div/div[1]/div[2]/div/form/div[3]/div')
search.submit()

# 2. In the above question, now scrape the following details of each product listed in first 3 pages of your search results and save it in a data frame and csv. In case if any product has less than 3 pages in search results then scrape all the products available under that product name. Details to be scraped are: "Brand Name", "Name of the Product", "Price", "Return/Exchange", "Expected Delivery", "Availability" and “Product URL”. In case, if any of the details are missing for any of the product then replace it by “-“.

In [52]:
all_urls=[]
pr_names=[]
for i in range(3):
    url=driver.find_elements(By.XPATH,"//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for u in url:
        all_urls.append(u.get_attribute('href'))
        
    pr_name=driver.find_elements(By.XPATH,"//a[@class='a-link-normal s-underline-text s-underline-link-text s-link-style a-text-normal']")
    for i in pr_name:
        if i.text is None:
            pr_names.append('--')
        else:
            pr_names.append(i.text)
            time.sleep(2)
    driver.find_element(By.XPATH,"//a[@class='s-pagination-item s-pagination-next s-pagination-button s-pagination-separator']").click()
    time.sleep(3)

print(len(all_urls))
print(len(pr_names))

70
70


In [54]:
Brand=[]
Price=[]
Return_Exchange=[]
Expected_Delivery=[] 
Availability=[]
for ur in all_urls:
    driver.get(ur)                                                        
    time.sleep(2)
    
    try:
        brand = driver.find_element(By.XPATH,'//a[@id="bylineInfo"]')      # Extracting Brand from xpath
        Brand.append(brand.text)
    except NoSuchElementException:
        Brand.append('-')
    
    try:
        price = driver.find_element(By.XPATH,"//div[@id='corePriceDisplay_desktop_feature_div']")            # Extracting Price from xpath
        Price.append(price.text)
    except NoSuchElementException:
        Price.append('-')
        
    try:                                                                                     # Extracting Return/Exchange policy from xpath
        ret = driver.find_element(By.XPATH,'//div[@data-name="RETURNS_POLICY"]/span/div[2]/a')
        Return_Exchange.append(ret.text)
    except NoSuchElementException:
        Return_Exchange.append('-')
    try:
        delivry = driver.find_element(By.XPATH,"//div[id='mir-layout-DELIVERY_BLOCK-slot-PRIMARY_DELIVERY_MESSAGE_LARGE']")         # Extracting Expected Delivery from xpath
        Expected_Delivery.append(delivry.text)
    except NoSuchElementException:
        Expected_Delivery.append('-')
     
    try:
        avl = driver.find_element(By.XPATH,'//div[@id="availability"]/span')                # Extracting Availability from xpath
        Availability.append(avl.text)
    except NoSuchElementException:
        Availability.append('-')
    
        time.sleep(3)
print(len(Brand))
print(len(Price))
print(len(Return_Exchange))
print(len(Expected_Delivery))
print(len(Availability))

70
70
70
70
70


In [56]:
df=pd.DataFrame({"URL":all_urls,"Product names":pr_names,"Brand":Brand,"Price":Price,"Return_Exchange":Return_Exchange,"Expected_Delivery":Expected_Delivery,"Availability":Availability})
df

,URL,Product names,Brand,Price,Return_Exchange,Expected_Delivery,Availability
0,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,"HP 15s, AMD Ryzen 5 5500U, 15.6 inch(39.6cm) F...",Brand: HP,"-12%\n₹43,790\nM.R.P.:\n₹50,000",-,-,In stock
1,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,Lenovo IdeaPad Slim 3 Intel Core i3 12th Gen 1...,Brand: Lenovo,"-37%\n₹41,990\nM.R.P.:\n₹66,390",-,-,In stock
2,https://www.amazon.in/Lenovo-Celeron-1920x1080...,"Lenovo V15 Intel Celeron N4500 15.6"" (39.62 cm...",Brand: Lenovo,"-43%\n₹23,999\nM.R.P.:\n₹42,440",-,-,In stock
3,https://www.amazon.in/Lenovo-E41-55-Windows-Gr...,Lenovo E41-55 Laptop (AMD Athlon Pro 3045B/ 4G...,Brand: Lenovo,"-60%\n₹18,595\nM.R.P.:\n₹46,480",-,-,In stock
4,https://www.amazon.in/AXL-Launched-Display-Win...,AXL Laptop (Vayu Book) Newly Launched Thin & L...,Visit the AXL Store,"-44%\n₹17,990\nM.R.P.:\n₹31,990",-,-,In stock
...,...,...,...,...,...,...,...
65,https://www.amazon.in/HP-Micro-Edge-Keyboard-G...,"HP 14s, 5th Gen AMD Ryzen 3-8GB RAM/512GB SSD ...",Brand: HP,"-19%\n₹38,990\nM.R.P.:\n₹48,294",-,-,In stock
66,https://www.amazon.in/ASUS-15-6-inch-Integrate...,"ASUS VivoBook 15 (2021), 15.6-inch (39.62 cm) ...",Visit the ASUS Store,"-21%\n₹26,990\nM.R.P.:\n₹33,990",-,-,In stock
67,https://www.amazon.in/HP-15-6-inches-Graphics-...,"HP 15s, 11th Gen Intel Core i3 8GB RAM/1TB HDD...",Brand: HP,"-24%\n₹38,990\nM.R.P.:\n₹51,470",-,-,In stock
68,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,"Honor MagicBook 15, AMD Ryzen 5 5500U 15.6-inc...",Visit the Honor Store,"-30%\n₹38,990\nM.R.P.:\n₹55,999",-,-,In stock


# 3. Write a python program to access the search bar and search button on images.google.com and scrape 10 images each for keywords ‘fruits’, ‘cars’ and ‘Machine Learning’, ‘Guitar’, ‘Cakes’. 

In [10]:
import pandas as pd

from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import time
import re
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By


import warnings
warnings.filterwarnings('ignore')



In [11]:
driver=webdriver.Chrome(r"chromedriver.exe")

In [12]:
driver.get("https://images.google.com/")

In [14]:
images=['fruits','cars','Machine Learning','guitars','cakes']     #searching items
links=[]                                                         # scraped url link list



    

In [15]:
# Scraping first 10 image sources of searched items and making dataframe
all_images=pd.DataFrame([])
all_images["Fruits"]=links
all_images["Cars"]=links
all_images["Machine Learning"]=links
all_images["Guitar"]=links
all_images["Cakes"]=links
all_images

,Fruits,Cars,Machine Learning,Guitar,Cakes


# 4.. Write a python program to search for a smartphone(e.g.: Oneplus Nord, pixel 4A, etc.) on www.flipkart.comand scrape following details for all the search results displayed on 1st page. Details to be scraped: “Brand Name”, “Smartphone name”, “Colour”, “RAM”, “Storage(ROM)”, “Primary Camera”, “Secondary Camera”, “Display Size”, “Battery Capacity”, “Price”, “Product URL”. Incase if any of the details is missing then replace it by “- “. Save your results in a dataframe and CSV.

In [71]:
#Python program for scraping mobile details in flipkart platform
driver=webdriver.Chrome(r"chromedriver.exe")
driver.get("http://www.flipkart.com/")     #for driver to get flipkart website
time.sleep(3)
driver.find_element(By.XPATH,"//button[@class='_2KpZ6l _2doB4z']").click() #closing login popup dialogue box

search_bar=driver.find_element(By.XPATH,"//input[@class='_3704LK']")   #Search field to send keys
search_bar.send_keys("one plus nord mobiles")
time.sleep(3)
search_button=driver.find_element(By.XPATH,"//button[@class='L0Z3Pu']")  #to click search item
search_button.submit()
time.sleep(3)

In [77]:
#scraping elements of modile details
Product_Nam=[]
Br=[]
Price=[]
Product_URL=[]
Battery=[]
Prim_Sec_camera=[]
Display=[]
RAM_ROM=[]
for i in range(3):
    URL=driver.find_elements(By.XPATH,"//a[@class='_1fQZEK']")
    #url=("//a[@class='_1fQZEK']")   #scraping mobile urls
    for i in url:
        Product_URL.append(i.get_attribute('href'))   #href links
time.sleep(3)

#Product names
product=x("//div[@class='_4rR01T']")
for i in product:
    Product_Nam.append(i.text)
#Scraping price 
price=x("//div[@class='_30jeq3 _1_WHN1']")
for i in price:
    Price.append(i.text)
#Scraping RAM_ROM details
ram_rom=x("//ul[@class='_1xgFaf']//li[1]")
for i in ram_rom:
    RAM_ROM.append(str(i.text).replace('|',' '))
#Scraping Display details
dis=x("//ul[@class='_1xgFaf']//li[2]")
for i in dis:
    Display.append(i.text)
#Scraping camera details
prim=x("//ul[@class='_1xgFaf']//li[3]")
for i in prim:
    Prim_Sec_camera.append(i.text)
#Scraping battery details
batt=x("//ul[@class='_1xgFaf']//li[4]")
for i in batt:
    Battery.append(i.text)

StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=113.0.5672.129)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00698893+48451]
	(No symbol) [0x0062B8A1]
	(No symbol) [0x00535058]
	(No symbol) [0x0053E1F3]
	(No symbol) [0x005386C8]
	(No symbol) [0x005386A6]
	(No symbol) [0x00537853]
	(No symbol) [0x00539DEB]
	(No symbol) [0x0058CDD9]
	(No symbol) [0x0057A2BC]
	(No symbol) [0x0058C482]
	(No symbol) [0x0057A0B6]
	(No symbol) [0x00557E08]
	(No symbol) [0x00558F2D]
	GetHandleVerifier [0x008F8E3A+2540266]
	GetHandleVerifier [0x00938959+2801161]
	GetHandleVerifier [0x0093295C+2776588]
	GetHandleVerifier [0x00722280+612144]
	(No symbol) [0x00634F6C]
	(No symbol) [0x006311D8]
	(No symbol) [0x006312BB]
	(No symbol) [0x00624857]
	BaseThreadInitThunk [0x760C7D59+25]
	RtlInitializeExceptionChain [0x77BBB74B+107]
	RtlClearBits [0x77BBB6CF+191]


# 5. Write a program to scrap geospatial coordinates (latitude, longitude) of a city searched on google maps. 

In [1]:
# Importing necessary libraries.

from selenium import webdriver
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import time
import re
import requests
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.by import By


import warnings
warnings.filterwarnings('ignore')

In [2]:
driver=webdriver.Chrome(r"chromedriver.exe")

In [3]:
driver.get("https://www.google.com/maps/")
time.sleep(3)

In [6]:
city=input("Enter the name of address :")     #input field to enter location
loc=driver.find_element(By.ID,"searchboxinput")  #send keys to the search field 
loc.send_keys(city)
time.sleep(3)
search=driver.find_element(By.XPATH,"/html/body/div[3]/div[9]/div[3]/div[1]/div[1]/div[1]/div[2]/div[1]/button")
search.click()                    #Click on search button
time.sleep(3)

Enter the name of address :GWALIOR


In [7]:
try:
    url_string=driver.current_url            #To get current url of location
    print("url_extracted :",url_string)
    lat_long=re.findall(r'@(.*)data',url_string)  #Extracting url 
    if len(lat_long):
        lat_long_list=lat_long[0].split(',')
        if len(lat_long_list)>=2:
            lat=lat_long_list[0]                   #Scraping latitude detail of url
            long=lat_long_list[1]                  #Scraping longitude detail of url
        print("Latitude = {}, Longitude = {}".format(lat, long)) 

except Exception as e:
    print("error :",str(e))

url_extracted : https://www.google.com/maps/place/Gwalior,+Madhya+Pradesh/@26.2140619,78.0260852,11z/data=!3m1!4b1!4m6!3m5!1s0x3976c5d1792291fb:0xff4fb56d65bc3adf!8m2!3d26.2124007!4d78.1772053!16zL20vMDE5Nl9o?entry=ttu
Latitude = 26.2140619, Longitude = 78.0260852


# 6.Write a program to scrap all the available details of best gaming laptops from digit.in

In [41]:
# Importing necessary libraries

import pandas as pd
import selenium
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import requests
import re
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [42]:
driver=webdriver.Chrome(r"chromedriver.exe")

In [43]:
driver.get("https://www.digit.in/")  
time.sleep(3)

In [44]:
# searching for best Laptop
best_gam_laptops = driver.find_element(By.XPATH,"//div[@class='listing_container']//ul//li[9]").click()


In [45]:
# creating empty list
Laptop_Name = []
Operating_sys = []
Display = []
Processor = []
Memory = []
Weight = []
Dimensions = []
Graph_proc = []
Price = []

In [51]:
#scraping the data of laptop names
laptop_name = driver.find_elements(By.XPATH,"//div[@class='right-container']/div/a/h3")
for name in laptop_name:
    Laptop_Name.append(name.text)
    
#scraping the data of operating system
try:
    op_sys = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[1]/div/div")
    for os in op_sys:
        Operating_sys.append(os.text)
except NoSuchElementException:
    pass


#scraping data of display of the Laptop
try:
    display = driver.find_elements(By.XPATH,"//div[@class='product-detail']/div/ul/li[2]/div/div")
    for disp in display:
        Display.append(disp.text)
except NoSuchElementException:
    pass


# scraping data of processor
try:
    processor = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[5]/td[3]")
    for pro in processor:
        Processor.append(pro.text)
except NoSuchElementException:
    pass


# scraping the data of memory
try:
    memory = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[6]/td[3]")
    for memo in memory:
        Memory.append(memo.text)
except NoSuchElementException:
    pass


# scraping data of weight
try:
    weight = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[7]/td[3]")
    for wgt in weight:
        Weight.append(wgt.text)
except NoSuchElementException:
    pass


# scraping data of dimensions
try:
    dimension = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[8]/td[3]")
    for dim in dimension:
        Dimensions.append(dim.text)
except NoSuchElementException:
    pass


# scraping data of graph processor
try:
    graph = driver.find_elements(By.XPATH,"//div[@class='Spcs-details'][1]/table/tbody/tr[9]/td[3]")
    for gra in graph:
        Graph_proc.append(gra.text)
except NoSuchElementException:
    pass


# scraping the data of price
try:
    price = driver.find_elements(By.XPATH,"//td[@class='smprice']")
    for pri in price:
        Price.append(pri.text.replace('₹ ','Rs'))
except NoSuchElementException:
    pass

In [54]:
print(len(Laptop_Name),
len(Operating_sys),
len(Display),
len(Processor),
len(Memory),
len(Weight),
len(Dimensions),
len(Price))


0 14 14 14 14 14 14 14


In [55]:
#creating DataFrame for scraped data
Gaming_Laptop = pd.DataFrame({})
Gaming_Laptop['Laptop Name'] = Laptop_Name
Gaming_Laptop['Operating System'] =Operating_sys
Gaming_Laptop['Display'] = Display
Gaming_Laptop['Processor'] = Processor
Gaming_Laptop['Memory'] = Memory
Gaming_Laptop['Weight'] = Weight
Gaming_Laptop['Dimensions'] = Dimensions
#Gaming_Laptop['Graphical Processor'] = Graph_proc
Gaming_Laptop['Price'] = Price
Gaming_Laptop

,Laptop Name,Operating System,Display,Processor,Memory,Weight,Dimensions,Price
0,NaN,Windows 11 Home,"17.3"" (2560 x 1440)",16 GB DDR5GB RAM & 1 TB SSD,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,397.1 x 262 x 27 mm dimension & 2.78 kg weight,"₹ 269,777","Rs269,777"
1,NaN,Windows 11 Home,"17.3"" (3840 x 2160)",64 GB DDR5GB RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080Ti Graphics ...,397 x 330 x 23 mm dimension & 3.3 kg weight,"₹ 499,990","Rs499,990"
2,NaN,Windows 11 Home,"16"" (2560 x 1600)",32 GB DDR5GB RAM & 1 TB SSD,NVIDIA GeForce RTX 3070 Ti Graphics card,359.9 x 264.4 x 19.9 mm dimension & 3.6 kg weight,"₹ 213,900","Rs213,900"
3,NaN,Windows 11 Home,"18"" (1920 x 1200)",32 GB DDR5GB RAM & 1 TB SSD,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,294 x 399 x 23 mm dimension & 3.1 kg weight,"₹ 279,990","Rs279,990"
4,NaN,Windows 11 Home,"16"" (2560 x 1600)",16 GB DDR5GB RAM & 1 TB SSD,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,360 x 279 x 28 mm dimension & 2.6 kg weight,"₹ 149,990","Rs149,990"
5,NaN,Windows 11 Home,"14"" (1920 x 1200)",16 GB DDR5GB RAM & 1 TB SSD,8 GB DDR6 AMD Radeon RX 6700S Graphics card,312 x 227 x 19 mm dimension & 1.65 kg weight,"₹ 156,990","Rs156,990"
6,NaN,Windows 11 Home,"15.6"" (1920 x 1080)",16 GB DDR5GB RAM & 1 TB SSD,8 GB DDR6 NVIDIA GeForce RTX 4060 Graphics card,& 1.98 kg weight,"₹ 130,990","Rs130,990"
7,NaN,Windows 11 Home,"17.3"" (2560 x 1440)",16 GB DDR5GB RAM & 1 TB SSD,12 GB DDR6 NVIDIA GeForce RTX 4080 Graphics card,397.1 x 262 x 27 mm dimension & 2.78 kg weight,"₹ 269,777","Rs269,777"
8,NaN,Windows 11 Home,"17.3"" (3840 x 2160)",64 GB DDR5GB RAM & 2 TB SSD,16 GB DDR6 NVIDIA GeForce RTX 3080Ti Graphics ...,397 x 330 x 23 mm dimension & 3.3 kg weight,"₹ 499,990","Rs499,990"
9,NaN,Windows 11 Home,"16"" (2560 x 1600)",32 GB DDR5GB RAM & 1 TB SSD,NVIDIA GeForce RTX 3070 Ti Graphics card,359.9 x 264.4 x 19.9 mm dimension & 3.6 kg weight,"₹ 213,900","Rs213,900"


# 7.Write a python program to scrape the details for all billionaires from www.forbes.com. Details to be scrapped: “Rank”, “Name”, “Net worth”, “Age”, “Citizenship”, “Source”, “Industry”.

In [17]:
import pandas as pd
import selenium
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import requests
import re
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [18]:
driver=webdriver.Chrome(r"chromedriver.exe")

In [19]:
# getting the specified url
url = "https://www.forbes.com/"
driver.get(url)

In [20]:
#let's get option button from the page
opt_btn = driver.find_elements(By.XPATH,"/html/body/div[1]/header/nav/div[1]/div[1]/div/div")


In [21]:
#select billionaires from options
blns = driver.find_elements(By.XPATH,"/html/body/div[1]/header/nav/div[3]/ul/li[1]")

In [22]:

#select world billionaire
bln_list = driver.find_elements(By.XPATH,"/html/body/div[1]/header/nav/div[3]/ul/li[1]/div[2]/ul/li[2]/a")

In [23]:
# scraping required data from the web page
# creating empty lists
Rank = []
Person_Name = []
Net_worth = []
Age = []
Citizenship = []
Source = []
Industry = []


for i in range(3):
    
    # scraping the data of rank of the billionaires
    rank_tag = driver.find_elements(By.XPATH,"//div[@class='Table_rank___YBhk Table_dataCell__2QCve']")
    for i in rank_tag:
        Rank.append(i.text)
    time.sleep(1)
    
    
 
    # scraping the data  of names of the billionaires
    name_tag = driver.find_elements(By.XPATH,"//div[@class='Table_dataCell__2QCve']")
    for i in name_tag:
        Person_Name.append(i.text)
    time.sleep(1)
    
    
    # scraping the data of age of the billionaires
    age_tag = driver.find_elements(By.XPATH,"//div[@class='Table_dataCell__2QCve']")
    for i in age_tag:
        Age.append(i.text)
    
    
    
    # scraping the data of citizenship of the billionaires
    cit_tag = driver.find_elements(By.XPATH,"//div[@class='TableRow_cell__db-hv Table_cell__houv9']")
    for i in cit_tag:
        Citizenship.append(i.text)
   
    
    
    # scraping the data of source of income of the billionaires
    sour_tag = driver.find_elements(By.XPATH,"//div[@class='TableRow_cell__db-hv Table_cell__houv9']")
    for i in sour_tag:
        Source.append(i.text)
    
    
    
    # scraping data of industry of the billionaires
    ind_tag = driver.find_elements(By.XPATH,"//div[@class='Table_dataCell__2QCve']")
    for i in ind_tag:
        Industry.append(i.text)
    
    
    # scraping data of net_worth of billionaires
    net_tag = driver.find_elements(By.XPATH,"//div[@class='Table_netWorth___L4R5 Table_dataCell__2QCve']")
    for i in net_tag:
        Net_worth.append(i.text)
    
    
    # clicking on next button
    #try:
       # next_button = driver.find_element(By.XPATH,"//button[@class='pagination-btn pagination-btn--next ']")
       ## next_button.click()
    #except:
        #break

In [24]:
print(len(Rank),
len(Person_Name),
len(Net_worth),
len(Age),
len(Citizenship),
len(Source),
len(Industry))

600 2400 600 2400 4200 4200 2400


In [26]:
# framing Data
Billionaires = pd.DataFrame({})
Billionaires['Rank'] = Rank[:500]
Billionaires['Name'] = Person_Name[:500]
Billionaires['Net Worth'] = Net_worth[:500]
Billionaires['Age'] = Age[:500]
Billionaires['Citizenship'] = Citizenship[:500]
Billionaires['Source'] = Source[:500]
Billionaires['Industry'] = Industry[:500]
Billionaires

,Rank,Name,Net Worth,Age,Citizenship,Source,Industry
0,1,Bernard Arnault & family,$211 B,Bernard Arnault & family,1,1,Bernard Arnault & family
1,2,74,$180 B,74,Bernard Arnault & family,Bernard Arnault & family,74
2,3,LVMH,$114 B,LVMH,$211 B,$211 B,LVMH
3,4,Fashion & Retail,$107 B,Fashion & Retail,74,74,Fashion & Retail
4,5,Elon Musk,$106 B,Elon Musk,France,France,Elon Musk
...,...,...,...,...,...,...,...
495,94,Diversified,$17.5 B,Diversified,Cheese,Cheese,Diversified
496,97,Aliko Dangote,$17.4 B,Aliko Dangote,Food & Beverage,Food & Beverage,Aliko Dangote
497,97,65,$17.4 B,65,72,72,65
498,99,"Cement, sugar",$17.1 B,"Cement, sugar",Abigail Johnson,Abigail Johnson,"Cement, sugar"


# 8.Write a program to extract at least 500 Comments, Comment upvote and time when comment was posted from any YouTube Video. 

In [164]:
import pandas as pd
import selenium
from bs4 import BeautifulSoup
import time
from selenium import webdriver
import requests
import re
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [165]:
driver=webdriver.Chrome(r"chromedriver.exe")

In [166]:
driver.get("https://www.youtube.com/results?search_query=most+viewed+english+song+on+youtube") #youtube vidio search for mo

In [167]:
try:
    driver.find_element(By.XPATH,"/html/body/ytd-app/div[1]/ytd-page-manager/ytd-search/div[1]/ytd-two-column-search-results-renderer/div/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-video-renderer[7]/div[1]/div/div[1]/div/h3/a/yt-formatted-string").click()
    driver.maximize_window()
    
    print("start scrolling to generate more images on the page...")
# 500 time we scroll down by 600 in order to generate more images on the website
    for _ in range(500):
        driver.execute_script("window.scrollBy(0,500)")   
    print("End of scrolling...")
    comments=[]
    Time_posted=[]
    comment_upvote=[]
    
    comm=driver.find_elements(By.XPATH,"//yt-formatted-string[@class='style-scope ytd-comment-renderer']") #Scraping comments
    for i in comm:
        comments.append(str(i.text).replace('\n',''))
    time=driver.find_elements(By.XPATH,"//yt-formatted-string[@class='published-time-text style-scope ytd-comment-renderer']") #Scraping post time
    for i in time:    
        Time_posted.append(str(i.text).replace('\n',''))
    vote=driver.find_elements(By.XPATH,"//ytd-comment-action-buttons-renderer[@class='style-scope ytd-comment-renderer']")  #Scraping comments upvote
    for i in vote:    
        comment_upvote.append(str(i.text).replace('\n',' '))    
    print(len(comments),len(Time_posted),len(comment_upvote))
    print(comments,Time_posted,comment_upvote)
except Exception as e:
    print("Error :",e)

start scrolling to generate more images on the page...
End of scrolling...
200 220 220
["this kid is married now.  That's how old we are", 'Eu escutava essa música o dia inteiro, tempo bom que não volta mais ', 'Um hino é um hino, né meus amigos?', 'Eu amo essa música. ', 'Amooo', '2010: baby baby baby 2015: sorry sorry sorry 2020: yummy yummy yummy', 'Marcou minha infância ', 'Amoo de mais ', 'Y aqui sigo enamorada de este niño ', 'Esta canción vivira para siempre ️', 'E incrível, O quanto eu fico nas nuvens, quando eu ouço Baby', '8 years vs 1 weekWell played Youtube', "Looks like we're getting older but this song will never get old ", 'Tantos años sin escucharla y ahora suena súper bien, tremenda obra de arte', 'Saudades de quando o nosso único problema era essa música….', 'Pasaron 13 años y sigo escuchando esta rolita ', 'Apaixonada por um adolescente que não é mais adolescentes', '10M likes - 10M dislikes "perfectly balanced" - Thanosss', "This song never gets old!!! It's 2023 and

In [168]:
#Making dataframe for 500 comment details
Youtube=pd.DataFrame([])
Youtube["Comments"]=comments[0:200]
Youtube["Posted Time"]=Time_posted[0:200]
Youtube["Comment UpVotes"]=comment_upvote[0:200]
Youtube

,Comments,Posted Time,Comment UpVotes
0,this kid is married now. That's how old we are,3 years ago,105K Reply
1,"Eu escutava essa música o dia inteiro, tempo b...",1 day ago,43 Reply
2,"Um hino é um hino, né meus amigos?",1 day ago,20 Reply
3,Eu amo essa música.,12 days ago,46 Reply
4,Amooo,5 hours ago,5 Reply
...,...,...,...
195,2023 ouvindo só as boas,7 days ago,34 Reply
196,I used to listen this songs when I was 8 years...,6 days ago,2 Reply
197,Who came back just because of YouTube rewind b...,4 years ago,206 Reply
198,"""The infectious chorus gets stuck in your head...",8 days ago,1 Reply


# 9.. Write a python program to scrape a data for all available Hostels from https://www.hostelworld.com/ in “London” location. You have to scrape hostel name, distance from city centre, ratings, total reviews, overall reviews, privates from price, dorms from price, facilities and property description.

In [169]:
import pandas as pd
import selenium
import time
from selenium import webdriver
import requests
import re
import warnings
warnings.filterwarnings('ignore')
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains

In [170]:
driver=webdriver.Chrome(r"chromedriver.exe")

In [171]:
driver.get("https://www.hostelworld.com/")
time.sleep(3)

In [176]:
search_field=driver.find_element(By.XPATH,"//input[@class='native-input body-1-reg']")
search_field.click()
time.sleep(3)

In [179]:
search_bar=driver.find_element(By.XPATH,"//input[@class='native-input body-1-reg']")
search_bar.send_keys("London")

In [186]:
search_go=driver.find_element(By.XPATH,"//button[@class='btn-content large-button']")
search_go.click()
time.sleep(3)

In [187]:
Hostel_Name=[]
Distance_from_city=[]
Rating=[]
Total_review=[]
Private_price=[]
Dorm_Prices=[]

In [188]:
name=driver.find_elements(By.XPATH,"//h2[@class='title title-6']")
dist=driver.find_elements(By.XPATH,"//span[@class='description']")
rat=driver.find_elements(By.XPATH,"//div[@class='score orange big']")
tot_re=driver.find_elements(By.XPATH,"//div[@class='reviews']")
pr_pr=driver.find_elements(By.XPATH,"//a[@class='prices']//div[1]//div")
dorm_pr=driver.find_elements(By.XPATH,"//a[@class='prices']//div[2]")

for i in name:
    Hostel_Name.append(i.text)
for i in dist:
    Distance_from_city.append(i.text)
for i in rat:
    if i.text is None:
        Rating.append("--")
    else:
        Rating.append(i.text)
for i in tot_re:
    Total_review.append(i.text)
for i in pr_pr:
    if i.text is None:
        Private_price.append("--")
    else:
        Private_price.append(i.text)
for i in dorm_pr:
    Dorm_Prices.append(i.text)

In [192]:
Hostel=pd.DataFrame({"Hostel Name":Hostel_Name,"Distance from city":Distance_from_city,"Total Review":Total_review,"Dorm prices":Dorm_Prices})
Hostel[:10]

,Hostel Name,Distance from city,Total Review,Dorm prices
0,Wombat's City Hostel London,Hostel - 3.6km from city centre,14849 Total Reviews,Dorms From\nRs6985.7 Rs5589
1,Onefam Notting Hill by Hostel One,Hostel - 5.5km from city centre,2030 Total Reviews,Dorms From\nRs7500
2,St Christopher's Village,Hostel - 1.8km from city centre,12130 Total Reviews,Dorms From\nRs3689.99 Rs3321
3,Astor Hyde Park,Hostel - 4.3km from city centre,12034 Total Reviews,Dorms From\nRs4002
4,Generator London,Hostel - 3km from city centre,7484 Total Reviews,Dorms From\nRs3706
5,Urbany Hostel London,Hostel - 5.4km from city centre,767 Total Reviews,Dorms From\nRs5963
6,NX London Hostel,Hostel - 6.1km from city centre,1876 Total Reviews,Dorms From\nRs3886.08 Rs3138
7,Safestay London Elephant & Castle,Hostel - 1.7km from city centre,4943 Total Reviews,Dorms From\nRs3640
8,Smart Hyde Park Inn Hostel,Hostel - 5km from city centre,6609 Total Reviews,Dorms From\nRs3392.06 Rs3053
9,Phoenix Hostel,Hostel - 4.2km from city centre,4110 Total Reviews,Dorms From\nRs3541.14 Rs3364
